In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from pyslfp import FingerPrint, plot_SHGrid, ResponseFields
import numpy as np

%load_ext autoreload
%autoreload 2

In [20]:
# Set some things up.
def inner_product(fp, f1, f2):
    g = fp.gravitational_acceleration
    b = fp.mean_sea_floor_radius
    return fp.integrate(f1.sl * f2.sl)  \
        - (1 /g) * fp.integrate((g * f1.u * f2.u + f1.phi * f2.phi))  \
        - (1 /g) * np.inner(f1.omega, f2.omega)

def inner_product2(fp, f1, f2):
    g = fp.gravitational_acceleration
    return fp.integrate(f1.sl * f2.sl + f1.u * f2.u + f1.phi * f2.phi) + np.inner(f1.omega, f2.omega)

# Set up the fingerprint solver. 
finger_print = FingerPrint(128)

# Set the background sea level and ice thickness. 
finger_print.set_background_state_from_ice_ng()

# Set the load. 
zeta1 = finger_print.northern_hemisphere_load()
zeta2 = finger_print.southern_hemisphere_load()
zero = zeta1 - zeta1

In [22]:
zeta1_u = zeta1 
zeta1_phi = zeta1 
kk1 = np.array([1,5]) 
zeta2_u = zeta2 
zeta2_phi = zeta2
kk2 = np.array([5,10]) 

# Invent a generalised load
generalised_load1 = ResponseFields(zeta1_u, zeta1_phi, kk1, zeta1)
generalised_load2 = ResponseFields(zeta2_u, zeta2_phi, kk2, zeta2)

# Compute the response. 
response1 = finger_print.generalised_solver(generalised_load1, rotational_feedbacks=True, verbose=True)
response2 = finger_print.generalised_solver(generalised_load2, rotational_feedbacks=True)

# Needs to be a full inner product (i.e. not just zeta*sl)
print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

# Plot the sea level change over the oceans. 
#plot_SHGrid(response1.sl * finger_print.ocean_mask() , contour=True, levels=60, colorbar=True)

Iteration = 1, relative error = 4.1623e-02
Iteration = 2, relative error = 1.9159e-03
Iteration = 3, relative error = 2.2733e-04
Iteration = 4, relative error = 3.6415e-05
Iteration = 5, relative error = 6.6809e-06
Iteration = 6, relative error = 1.2976e-06
Iteration = 7, relative error = 2.5910e-07
-1.1310713019522564e+20
-1.1310713019522525e+20


In [62]:
## Check to see if generalised solver is consistent with the normal solver.

generalised_load1 = ResponseFields(zero, zero, np.array([0,0]), zeta1)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.solver(zeta1)

print(finger_print.integrate(response1.sl * zeta1))
print(finger_print.integrate(response2.sl * zeta1))

2.6831278712261214e+20
2.6831278712261214e+20


In [5]:
## Try u only

generalised_load1 = ResponseFields(zeta1, zero, np.array([0,0]), zero)
generalised_load2 = ResponseFields(zeta2, zero, np.array([0,0]), zero)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

-1.5516739250868074e+19
-1.551673924583832e+19


In [6]:
## Try phi only

generalised_load1 = ResponseFields(zero, zeta1, np.array([0,0]), zero)
generalised_load2 = ResponseFields(zero, zeta2, np.array([0,0]), zero)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

5.950552193828723e+19
5.9505522057528435e+19


In [7]:
## Try zeta only

generalised_load1 = ResponseFields(zero, zero, np.array([0,0]), zeta1)
generalised_load2 = ResponseFields(zero, zero, np.array([0,0]), zeta2)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

-1.3615261520789651e+19
-1.361503064448356e+19


In [8]:
## Try k only

generalised_load1 = ResponseFields(zero, zero, np.array([1e25,2e25]), zero)
generalised_load2 = ResponseFields(zero, zero, np.array([5e25,3e25]), zero)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

822412556825230.6
822412556825230.6
